In [1]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import f1_score, accuracy_score
from sklearn.model_selection import StratifiedKFold
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import RFE, SelectFromModel
import optuna.integration.sklearn

In [2]:
all_new = pd.read_csv('data/feature_df_agg_svd16.csv')
BERT_svd256 = pd.read_csv("data/BERT_svd256.csv")
ALBERT_svd256 = pd.read_csv("data/ALBERT_svd256.csv")
ROBERTA_svd256 = pd.read_csv("data/ROBERTA_svd256.csv")
XLMROBERTA_svd256 = pd.read_csv("data/XLMROBERTA_svd256.csv")
tfidf_svd_raw_128 = pd.read_csv("data/tfidf_svd_raw_128_2gram.csv")
mfw_df = pd.read_csv('data/mfw.csv')
miw_df = pd.read_csv('data/miw.csv')

In [3]:
all_new = pd.merge(all_new, BERT_svd256, on="id", how="outer")
all_new = pd.merge(all_new, ALBERT_svd256, on="id", how="outer")
all_new = pd.merge(all_new, ROBERTA_svd256, on="id", how="outer")
all_new = pd.merge(all_new, XLMROBERTA_svd256, on="id", how="outer")
all_new = pd.merge(all_new, tfidf_svd_raw_128, on="id", how="outer")
all_new = pd.merge(all_new, miw_df, on="id", how="outer")
all_new = pd.merge(all_new, mfw_df, on="id", how="outer")

In [4]:
test_set = all_new[all_new["data_type"] == "test"]
train_set= all_new[all_new["data_type"] == "train"]

In [5]:
def run_optuna_random_forest(test, train, target_cols, feature_cols, seed):
    """
    import optuna.integration.sklearn
    """
    
    X_train = train[feature_cols]
    y_train = train[target_cols]
    X_test = test[feature_cols]
    
    y_preds = []
    models = []
    oof_train = np.zeros((len(X_train),))
    importances = []
    
    cv = StratifiedKFold(n_splits=7, shuffle=True, random_state=seed)
    for fold_id, (train_index, valid_index) in enumerate(cv.split(X_train, X_train["category2"])):
        X_tr = X_train.loc[train_index]
        X_val = X_train.loc[valid_index]
        y_tr = y_train.loc[train_index].values.ravel()
        y_val = y_train.loc[valid_index].values.ravel()

        clf = RandomForestClassifier(n_jobs=-1)
        param_distributions = {"n_estimators": optuna.distributions.IntUniformDistribution(50, 1000),
                               "max_depth": optuna.distributions.IntUniformDistribution(4, 50),
                               'min_samples_split': optuna.distributions.UniformDistribution(0.0001, 1.0),
                               'min_samples_leaf': optuna.distributions.IntUniformDistribution(1, 60),
                              }
        optuna_search = optuna.integration.OptunaSearchCV(clf,
                                                          param_distributions,
                                                          n_jobs=1,
                                                          n_trials=50,
                                                          )
        optuna_search.fit(X_tr, y_tr)
        
        oof_train[valid_index] = optuna_search.predict_proba(X_val)[:,1]
        y_pred = optuna_search.predict_proba(X_test)[:,1]

        y_preds.append(y_pred)
        models.append(optuna_search)
        
        # display importance
        importance = pd.DataFrame(optuna_search.best_estimator_.feature_importances_, index=feature_cols, columns=['importance'])
        display(importance)
        importances.append(importance)

    return oof_train, sum(y_preds) / len(y_preds), importances

In [6]:
def optuna_feature_selection(test, train, target_cols, feature_cols):
    X_train = train[feature_cols]
    y_train = train[target_cols].values.ravel()
    X_test = test[feature_cols]
    
    estimator = RandomForestClassifier(n_estimators=200,
                                       max_depth=40,
                                       min_samples_split=2,
                                       min_samples_leaf=30,
                                       n_jobs=-1)
    selector = RFE(estimator,
                   step=0.1,
                  verbose=True)
    
    param_distributions = {"n_features_to_select": optuna.distributions.IntUniformDistribution(100, 500)}
    optuna_search = optuna.integration.OptunaSearchCV(selector,
                                                      param_distributions,
                                                      n_jobs=1,
                                                      n_trials=50,
                                                      )
    optuna_search.fit(X_train, y_train)
    
    return optuna_search.best_estimator_.n_features_, optuna_search.best_estimator_.ranking_

In [7]:
def rfe_feature_selection(test, train, target_cols, feature_cols):
    X_train = train[feature_cols]
    y_train = train[target_cols].values.ravel()
    X_test = test[feature_cols]
    imp = SimpleImputer(missing_values=np.nan, strategy='mean')
    imp_train = imp.fit(X_train)
    imp_test = imp.fit(X_test)
    X_train = pd.DataFrame(imp_train.transform(X_train))
    X_test = pd.DataFrame(imp_test.transform(X_test))
    
    """
        estimator = RandomForestClassifier(n_estimators=200,
                                       max_depth=40,
                                       min_samples_split=2,
                                       min_samples_leaf=30,
                                       n_jobs=4)
            selector = RFE(estimator,
                   step=0.2,
                  verbose=True,
                  n_features_to_select=300,
                  )
    """
    
    estimator = GradientBoostingClassifier()
    selector = SelectFromModel(estimator)

    
    selector.fit(X_train, y_train)
    #print(selector.score(X_train, y_train))
    
    feature_arr = np.array(feature_cols)
    return list(feature_arr[selector.get_support()])

In [8]:
if __name__ == "__main__":
    
    with open("data/base_feature_config_lgmb.json") as f:
        feature_config = json.load(f)
    base_features = feature_config["selected_feature"]
    base_features.extend(list(mfw_df.columns.drop("id")))
    base_features.extend(list(miw_df.columns.drop("id")))
    
    target_cols = ["state"]
    
    additional_features = [
        (list(BERT_svd256.columns.drop("id")), "BERT"),
        (list(ALBERT_svd256.columns.drop("id")), "ALBERT"),
        (list(ROBERTA_svd256.columns.drop("id")), "ROBERTA"),
        (list(XLMROBERTA_svd256.columns.drop("id")), "XLMROBERTA"),
        (list(tfidf_svd_raw_128.columns.drop("id")), "tfidf"),
        (list(tfidf_svd_raw_128.columns.drop("id")) + list(BERT_svd256.columns.drop("id")), "tfidf+BERT"),
        (list(tfidf_svd_raw_128.columns.drop("id")) + list(ALBERT_svd256.columns.drop("id")), "tfidf+ALBERT"),
        (list(tfidf_svd_raw_128.columns.drop("id")) + list(ROBERTA_svd256.columns.drop("id")), "tfidf+ROBERTA"),
        (list(tfidf_svd_raw_128.columns.drop("id")) + list(XLMROBERTA_svd256.columns.drop("id")), "tfidf+XLMROBERTA"),
    ]
    
    for n, additional_feature in enumerate(additional_features):
        additional_feature, feature_name = additional_feature
        print(f"============ {feature_name} optuna serching ============")
        feature_cols = base_features + additional_feature
        
        oof, preds, importances = run_optuna_random_forest(test_set,
                                                           train_set,
                                                           target_cols,
                                                           feature_cols,
                                                           seed=n*2023,
                                                          )
        
        
        config_dict = {
        "selected_feature": feature_cols,
        "out_of_fold": list(oof),
        "prediction": list(preds),
        "seed": n*2023
        }
        
        with open(f"data/rforest_{feature_name}_seed2023.json", 'w') as f:
            json.dump(config_dict, f)

/home/teppei/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_split.py:670: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=7.
  warnings.warn(("The least populated class in y has only %d"
<ipython-input-5-87d9572ffdd0>:28: ExperimentalWarning: OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in the future.
  optuna_search = optuna.integration.OptunaSearchCV(clf,
/home/teppei/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:67: FutureWarning: Pass classifier=True as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "
[I 2021-01-31 23:56:49,503] A new study created in memory with name: no-name-9dbf60b5-b7cc-481a-95c4-88ad224f650e


============ BERT optuna serching ============


[I 2021-01-31 23:56:51,647] Trial 0 finished with value: 0.5038725384566412 and parameters: {'n_estimators': 229, 'max_depth': 27, 'min_samples_split': 0.6508992109955515, 'min_samples_leaf': 18}. Best is trial 0 with value: 0.5038725384566412.
[I 2021-01-31 23:57:09,164] Trial 1 finished with value: 0.7498351420973501 and parameters: {'n_estimators': 831, 'max_depth': 43, 'min_samples_split': 0.03046917667987886, 'min_samples_leaf': 55}. Best is trial 1 with value: 0.7498351420973501.
[I 2021-01-31 23:57:15,684] Trial 2 finished with value: 0.7195188573443492 and parameters: {'n_estimators': 514, 'max_depth': 12, 'min_samples_split': 0.18479767249625664, 'min_samples_leaf': 5}. Best is trial 1 with value: 0.7498351420973501.
[I 2021-01-31 23:57:17,716] Trial 3 finished with value: 0.5038725384566412 and parameters: {'n_estimators': 486, 'max_depth': 12, 'min_samples_split': 0.9979189384302036, 'min_samples_leaf': 23}. Best is trial 1 with value: 0.7498351420973501.
[I 2021-01-31 23:57

,importance
country,0.000467
duration,0.000975
category2,0.001995
number_of_<i>,0.000953
number_of_</i>,0.000686
...,...
BERT_svd_251,0.001032
BERT_svd_252,0.001387
BERT_svd_253,0.001175
BERT_svd_254,0.001175


<ipython-input-5-87d9572ffdd0>:28: ExperimentalWarning: OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in the future.
  optuna_search = optuna.integration.OptunaSearchCV(clf,
/home/teppei/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:67: FutureWarning: Pass classifier=True as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "
[I 2021-02-01 00:04:10,255] A new study created in memory with name: no-name-40d9dab5-7e33-4825-a002-0de1da65c63f
[I 2021-02-01 00:04:12,816] Trial 0 finished with value: 0.6980522525478595 and parameters: {'n_estimators': 333, 'max_depth': 39, 'min_samples_split': 0.6180869171975112, 'min_samples_leaf': 4}. Best is trial 0 with value: 0.6980522525478595.
[I 2021-02-01 00:04:15,269] Trial 1 finished with value: 0.5064173371989951 and parameters: {'n_estimators': 606, 'max_depth': 31, 'min_samples_split': 0.

...

,importance
country,0.000281
duration,0.000775
category2,0.001527
number_of_<i>,0.000674
number_of_</i>,0.000720
...,...
XLMROBERTA_svd_251,0.000746
XLMROBERTA_svd_252,0.000719
XLMROBERTA_svd_253,0.000831
XLMROBERTA_svd_254,0.000929


<ipython-input-5-87d9572ffdd0>:28: ExperimentalWarning: OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in the future.
  optuna_search = optuna.integration.OptunaSearchCV(clf,
/home/teppei/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:67: FutureWarning: Pass classifier=True as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "
[I 2021-02-01 07:42:35,940] A new study created in memory with name: no-name-b8d97145-8622-4ba7-b1b8-f1f34b33482e
[I 2021-02-01 07:42:42,193] Trial 0 finished with value: 0.7049456146451117 and parameters: {'n_estimators': 813, 'max_depth': 29, 'min_samples_split': 0.4947538425421689, 'min_samples_leaf': 46}. Best is trial 0 with value: 0.7049456146451117.
[I 2021-02-01 07:42:47,817] Trial 1 finished with value: 0.7060524215073143 and parameters: {'n_estimators': 751, 'max_depth': 14, 'min_samples_split': 0

,importance
country,0.000171
duration,0.000724
category2,0.001294
number_of_<i>,0.000694
number_of_</i>,0.000778
...,...
XLMROBERTA_svd_251,0.000281
XLMROBERTA_svd_252,0.000263
XLMROBERTA_svd_253,0.000297
XLMROBERTA_svd_254,0.000316


<ipython-input-5-87d9572ffdd0>:28: ExperimentalWarning: OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in the future.
  optuna_search = optuna.integration.OptunaSearchCV(clf,
/home/teppei/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:67: FutureWarning: Pass classifier=True as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "
[I 2021-02-01 07:50:47,915] A new study created in memory with name: no-name-096e9aa4-fb49-4749-a220-229eca7c16ed
[I 2021-02-01 07:51:01,413] Trial 0 finished with value: 0.7621404714213653 and parameters: {'n_estimators': 441, 'max_depth': 26, 'min_samples_split': 0.018690950521565376, 'min_samples_leaf': 25}. Best is trial 0 with value: 0.7621404714213653.
[I 2021-02-01 07:51:10,909] Trial 1 finished with value: 0.7395718347037822 and parameters: {'n_estimators': 465, 'max_depth': 37, 'min_samples_split':

,importance
country,0.000372
duration,0.000872
category2,0.001696
number_of_<i>,0.000729
number_of_</i>,0.000668
...,...
XLMROBERTA_svd_251,0.001044
XLMROBERTA_svd_252,0.001051
XLMROBERTA_svd_253,0.001015
XLMROBERTA_svd_254,0.001142
